In [ ]:
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import csv
import pandas as pd
from pandas.api.types import CategoricalDtype
import scipy as sp
from pathlib import Path
import tkinter as tk
from tkinter import filedialog
import itertools
from colour import Color
from datetime import date
import statsmodels.api as sm
from statsmodels.stats.weightstats import ztest as ztest
from sklearn.metrics import r2_score as R2S
import cairo

## Names and Palette ##

In [2]:
today = date.today()
print("Today's date:", today)
date = str(today)
date

Today's date: 2024-07-02


'2024-07-02'

In [11]:
tubulin = '[Tubulin] (' r'$\mu$' 'M)'
tub = 'tub'


DCXconc = '[DCX] (nM)' 
EBconc = '[EB] (nM)'
DCX = 'DCX'
Type = 'DCX Type'


drug = '[Drug] ' r'$(\mu M)$'
drugtype = 'Drug Type'


Concentration = 'Concentration (' r'$\mu$' 'M)'
Length = 'Length (' r'$\mu$' 'm)'
Lifetime = 'Lifetime (min)'
GrowthRate = 'Growth Rate (' r'$\mu$' 'm/min)'
TimeToNucleate = 'Time to Nucleate (min)'
ShrinkageLength = 'Shrink Length (' r'$\mu$' 'm)'
ShrinkageLifetime = 'Shrink Lifetime (min)'
ShrinkageRate = 'Shrink Rate (' r'$\mu$' 'm/min)'
RescuePerLength = 'Rescues (' r'$\mu$' 'm' r'$^{-1}$' ')'

In [4]:
Type_order = ['None','WT', 'Y64N','R76S','R102S', 'R178L', 'P191R', 'G223E','R303X']

category_Type = CategoricalDtype(Type_order, ordered=True,) #order mutants

In [5]:
grey00 = '#777777'
grey20 = '#C5C5C5'
cyan00 = '#0089CF'  
cyan20 = '#65CBFF'
pink00 = '#e468b8'
pink20 = '#FBB0E1'
orange00 = '#ff8341'
orange20 = '#FFC6A8'


In [6]:
black = '#000000'
grey = '#777777'
blue = '#23459C'
cyan = '#0089CF'
green = '#79BA1A'
purple = '#AB4A9C'
lily = '#CB81EA'
magenta = '#D80B8C'
red = '#e60000'
orange = '#FF7300'
darkorange = '#FF5B00'
lightorange = '#FFAC83'
yellow = '#FFCB05'
lightyellow = '#F9DC2D'
pink = '#FFA4CD'
StartHue1 = Color(purple)
DC1color = list(StartHue1.range_to(Color(pink),3))
StartHue2 = Color(darkorange)
DC2color = list(StartHue2.range_to(Color(lightorange),3))
print(DC1color,DC2color)

palette_all = [grey] + list(map(Color.get_hex,DC1color)) + list(map(Color.get_hex,DC2color)) + [green,cyan]

parameters = [GrowthRate,TimeToNucleate,Lifetime,ShrinkageRate]

[<Color #ab4a9c>, <Color #e468b8>, <Color #ffa4cd>] [<Color #ff5b00>, <Color #ff8341>, <Color #ffac83>]


In [7]:
### font for matplotlib
matplotlib.rcParams['font.family'] = 'Helvetica Neue LT Std'

## Open Master Sheet Dynamics ##

In [8]:
askmastersheet = filedialog.askopenfilename() # show an "Open" dialog box and return the path to the selected file
data = pd.read_csv(askmastersheet, encoding='utf-8')
data[Type] = data[Type].fillna('None')
data 

,Date,Investigator,CH,[Tubulin] ($\mu$M),[DCX] (nM),Length ($\mu$m),Lifetime (min),Growth Rate ($\mu$m/min),Time to Nucleate (min),Shrink Length ($\mu$m),Shrink Lifetime (min),Shrink Rate ($\mu$m/min),Rescues,DCX Type
0,2020-07-03,Sofia,13,12,0,3.959000,4.024000,0.983847,0.901208,4.066000,0.083833,48.500994,False,None
1,2020-07-03,Sofia,13,12,0,7.490000,7.880333,0.950467,0.146708,7.918000,0.167667,47.224652,False,None
2,2020-07-03,Sofia,13,12,0,3.210000,3.521000,0.911673,0.230542,3.424000,0.251500,13.614314,False,None
3,2020-07-03,Sofia,13,12,0,12.840000,14.335500,0.895679,0.817375,12.947000,1.089833,11.879798,False,None
4,2020-07-03,Sofia,13,12,0,3.424000,3.940167,0.868999,0.649708,3.210000,0.251500,12.763419,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11303,2021-01-29,Sofia,22,6,227,3.729635,6.715666,0.555363,0.605930,0.845384,0.474046,1.783338,True,WT
11304,2021-01-29,Sofia,22,6,227,6.115573,10.743261,0.569247,5.305802,1.044122,0.789946,1.321763,True,WT
11305,2021-01-29,Sofia,22,6,227,4.376856,8.415779,0.520077,0.645753,1.342899,0.355596,3.776469,True,WT
11306,2021-01-29,Sofia,22,6,227,8.121382,16.386184,0.495624,0.530277,4.733321,2.453969,1.928843,True,WT


In [25]:
pathmastersheet = Path(askmastersheet)
pathmastersheet

WindowsPath('Z:/Shared/2024 Cruz Kelly Hendricks Brouhard/Sofia/Datasheets-and-Plots/2022-08-17_dynamics_MasterSheet.csv')

In [57]:
data = data.dropna()
data[Type] = data[Type].astype(category_Type)

In [69]:
data =  data[((data[tubulin]== 6) & (data[Type]!='R303X') & (data[Type]!= 'R102S'))]

## Show Where Data came from ##

In [70]:
data_source = data.groupby(by=[tubulin, Type, DCXconc,'Date','CH'],observed=True)[["Investigator"]].count().sort_values(by=[Type, DCXconc])

In [71]:
data_source = data_source.rename(columns={"Investigator": "n"})
data_source

n
[Tubulin] ($\mu$M) DCX Type [DCX] (nM) Date       CH    
6                  None     0          2020-07-03 22  13
                                       2020-07-20 13  64
                                       2020-07-21 11  21
                                       2020-07-29 13  57
                                       2020-08-03 12  61
...                                                   ..
                   G223E    114        2021-01-14 13  87
                                                  24  87
                            227        2021-01-14 11  57
                                                  23  69
                                       2021-01-18 21  89

[117 rows x 1 columns]

In [74]:
name = '2022-08-17_dynamics_MasterSheet_ThesisCH2-paper.csv'
name

'2022-08-17_dynamics_MasterSheet_ThesisCH2-paper.csv'

In [75]:
directorymastersheet = str(pathmastersheet.parents[0])
path = Path(directorymastersheet)
data.to_csv(path/(name), encoding='utf-8', index=True)
path

WindowsPath('Z:/Shared/2024 Cruz Kelly Hendricks Brouhard/Sofia/Datasheets-and-Plots')

## Open Master Sheet Curvature ##

In [20]:
askmastersheet = filedialog.askopenfilename() # show an "Open" dialog box and return the path to the selected file
data = pd.read_csv(askmastersheet, encoding='utf-8')
data[Type] = data[Type].fillna('None')
data 

,Curve Name,Curve Length (um),Average Curvature (um-1),Curvature Std (um-1),X-Coordinate (um),Y-Coordinate (um),Point Curvature (um-1),Point Curvature Sign,Red Intensity,Green Intensity,Blue Intensity,Date,CH,[DCX] (nM),DCX Type,Drug Type,[Drug] ($\mu$M),Filename
0,CURVE 1,27.108929,0.333002,0.569584,35.004900,36.460800,0.018782,-1,153,68,68,2023-04-05,5,5,WT,Paclitaxel,10,2-Stack_final_amplified
1,CURVE 1,27.108929,0.333002,0.569584,34.981278,36.466700,0.018370,-1,170,17,17,2023-04-05,5,5,WT,Paclitaxel,10,2-Stack_final_amplified
2,CURVE 1,27.108929,0.333002,0.569584,34.957887,36.472531,0.017925,-1,170,17,17,2023-04-05,5,5,WT,Paclitaxel,10,2-Stack_final_amplified
3,CURVE 1,27.108929,0.333002,0.569584,34.934724,36.478295,0.017448,-1,204,0,0,2023-04-05,5,5,WT,Paclitaxel,10,2-Stack_final_amplified
4,CURVE 1,27.108929,0.333002,0.569584,34.911788,36.483991,0.016935,-1,204,0,0,2023-04-05,5,5,WT,Paclitaxel,10,2-Stack_final_amplified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537467,CURVE 8,11.242575,0.664717,0.776231,53.554690,14.204073,0.004750,-1,1632,170,459,2023-05-16,1,5,WT,Sir-Tubulin,10,6-Stack_Composite-amplified-final
537468,CURVE 8,11.242575,0.664717,0.776231,53.485662,14.237860,0.004480,-1,4369,0,357,2023-05-16,1,5,WT,Sir-Tubulin,10,6-Stack_Composite-amplified-final
537469,CURVE 8,11.242575,0.664717,0.776231,53.415586,14.272129,0.004224,-1,5389,0,425,2023-05-16,1,5,WT,Sir-Tubulin,10,6-Stack_Composite-amplified-final
537470,CURVE 8,11.242575,0.664717,0.776231,53.344454,14.306886,0.003983,-1,4845,0,476,2023-05-16,1,5,WT,Sir-Tubulin,10,6-Stack_Composite-amplified-final


In [21]:
pathmastersheet = Path(askmastersheet)
pathmastersheet

WindowsPath('C:/Users/Brouhard Lab/OneDrive - McGill University/CurrBio2023/Sofia/Data/curvature-intensity/jointdata-on_2023-06-13_MasterSheet.csv')

In [10]:
data = data.dropna()
data[Type] = data[Type].astype(category_Type)

## Show Where Data came from ##

In [30]:
data_source = data.groupby(by=[ 'Date',Type, drugtype,'CH'],observed=True)[['Filename']].count().sort_values(by=[Type,drugtype])

In [31]:
data_source = data_source.rename(columns={"Filename": "n"})
data_source

n
Date       DCX Type Drug Type    CH       
2023-04-11 R178L    Paclitaxel   4   25472
2023-04-25 R178L    Paclitaxel   1   21248
                                 2   23168
2023-05-08 R89G     Paclitaxel   1   33280
                                 2   33152
                                 3   31616
2023-05-11 WT       Docetaxel    1   36224
                                 2   43648
                                 3   41216
2023-05-12 WT       Epothilone D 1   28800
                                 2   37888
                                 3   39552
2023-04-05 WT       Paclitaxel   5   21248
2023-04-11 WT       Paclitaxel   1   34176
                                 2   31872
2023-05-10 WT       Sir-Tubulin  1    6144
                                 2   18688
2023-05-16 WT       Sir-Tubulin  1   30080

In [74]:
name = '2022-08-17_dynamics_MasterSheet_ThesisCH2-paper.csv'
name

'2022-08-17_dynamics_MasterSheet_ThesisCH2-paper.csv'

In [75]:
directorymastersheet = str(pathmastersheet.parents[0])
path = Path(directorymastersheet)
data.to_csv(path/(name), encoding='utf-8', index=True)
path

WindowsPath('Z:/Shared/2024 Cruz Kelly Hendricks Brouhard/Sofia/Datasheets-and-Plots')